In [18]:
from typing import Any , Dict
from google.adk.agents import Agent , LlmAgent
from google.adk.apps.app import App , EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService, InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types

In [49]:
#Agent Executor Function

async def agent_invoke(
        agent: Runner,
        user_input: list[str] | str = None,
        session_name :str = "default_session",
):
    
   
    app_name = agent.app_name

    session = await session_service.get_session(app_name = app_name, session_id = session_name, user_id = USER_ID)

    if session:
        print(f"Resuming session: {session_name}")
    else:
        session = await session_service.create_session(app_name = app_name, session_id = session_name, user_id = USER_ID)
        print(f"creating new session: {session_name}")

    if user_input:
        if type(user_input) == str:
            user_input = [user_input]

        for msg in user_input:
            print(f"user: {msg}")

            msg = types.Content(role="user", parts=[types.Part(text = msg)])    

            async for event in runner.run_async(user_id = USER_ID , session_id = session.id , new_message = msg):
                if event.content and event.content.parts:
                    print(f"model: {event.content.parts[0].text}")
    else:
        print(f"No querries found.")

In [26]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

In [39]:

root_agent = LlmAgent(
    name= "Chatbot_Agent",
    model= Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="A memory based Chatbot."
)

print("Agent Created.")

Agent Created.


In [40]:
APP_NAME = "agentic_ai_03"
USER_ID = "default"
SESSION = "default"

session_service = InMemorySessionService()

runner = Runner(
    agent= root_agent,
    app_name= APP_NAME,
    session_service= session_service
)

App name mismatch detected. The runner is configured with app name "agentic_ai_03", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [ ]:
result = await agent_invoke(
    runner,
    ["Hello my name is Aamish.? I want to learn Machine learning.",
     "can you help me for building a roadmap.",
     "what is my name?"],

    "Aamish_Session"
)

Now we Use DBMS to save our Memory

In [47]:
APP_NAME = "agents"
USER_ID = "default"
SESSION = "default"
db_url = "sqlite:///Agentic_AI_03.db"

session_service = DatabaseSessionService(db_url= db_url)

runner = Runner(
    agent= root_agent,
    app_name= APP_NAME,
    session_service= session_service
)

print("Agent with DBMS Created.")

Agent with DBMS Created.


In [ ]:
result = await agent_invoke(
    runner,
    ["Hello my name is Aamish.? I want to learn Machine learning.",
     "can you help me for building a roadmap.",
     "what is my name?"],

    "Aamish_session_DB_02"
)